**We need to get the latitude and the longitude coordinates of each neighborhood to be added to df_toronto that we already built in the previous notebook so we are going to repeat some steps**

**Extracting the table again**

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wiki_page= requests.get(wiki_link).text

soup = BeautifulSoup(raw_wiki_page,'lxml')

In [2]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [3]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_T      = []
Borough_T       = []
Neighbourhood_T = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_T.append(p_var)
    Borough_T.append(b_var)
    Neighbourhood_T.append(n_var)

num of unique Postal codes: 84


**Creating the df_toronto**

In [9]:
df_toronto = pd.DataFrame({'Borough':Borough_T,'Neighbourhood':Neighbourhood_T,'Postalcode':Postcode_T,},columns=['Postalcode','Borough','Neighbourhood',])
df_toronto.head(15)

,Postalcode,Borough,Neighbourhood
0,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
1,M2M,North York,"Newtonbrook, Willowdale"
2,M1V,Scarborough,"Agincourt North, Milliken"
3,M5E,Downtown Toronto,Berczy Park
4,M6B,North York,Glencairn
5,M6H,West Toronto,Dovercourt Village
6,M3H,North York,"Bathurst Manor, Wilson Heights"
7,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill"
8,M5L,Downtown Toronto,Commerce Court
9,M5P,Central Toronto,Forest Hill North


**Extracting Latitude and Longtitude for each Postal code in Toronto and concating them to our data frame**

In [11]:
latLong = pd.read_csv('http://cocl.us/Geospatial_data')
latLong = latLong.astype(object)
new = pd.concat([df_toronto, latLong], axis = 1, join = 'inner')
df_toronto = new.copy()

In [12]:
df_toronto.head(12)


,Postalcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1B,43.8067,-79.1944
1,M2M,North York,"Newtonbrook, Willowdale",M1C,43.7845,-79.1605
2,M1V,Scarborough,"Agincourt North, Milliken",M1E,43.7636,-79.1887
3,M5E,Downtown Toronto,Berczy Park,M1G,43.771,-79.2169
4,M6B,North York,Glencairn,M1H,43.7731,-79.2395
5,M6H,West Toronto,Dovercourt Village,M1J,43.7447,-79.2395
6,M3H,North York,"Bathurst Manor, Wilson Heights",M1K,43.7279,-79.262
7,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill",M1L,43.7111,-79.2846
8,M5L,Downtown Toronto,Commerce Court,M1M,43.7163,-79.2395
9,M5P,Central Toronto,Forest Hill North,M1N,43.6927,-79.2648


In [14]:
df_toronto.to_csv('toronto_base.csv')